## Попытка решить как задачу классификации 
(но кластер лишь один)

In [99]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder
from skmultilearn.ensemble import LabelSpacePartitioningClassifier
from skmultilearn.problem_transform import LabelPowerset
from community import community_louvain
import networkx as nx

# 1. Кастомный кластеризатор для фиксированных кластеров
class FixedClusterer:
    def __init__(self, clusters):
        self.clusters = clusters
        
    def fit_predict(self, X, y):
        return self.clusters


# 1. Загрузка и подготовка данных
def load_data(filename):
    data = pd.read_feather(filename)
    
    # Преобразование в бинарные метки (топ-3 кода)
    y = pd.DataFrame(np.zeros((len(data), 6), dtype=bool), columns=[f'HL_{i+1}' for i in range(6)])
    for i, row in data.iterrows():
        top3 = row[['HL_1', 'HL_2', 'HL_3', 'HL_4', 'HL_5', 'HL_6']].nlargest(3).index
        y.loc[i, top3] = True
    
    X = data.drop(columns=['HL_1', 'HL_2', 'HL_3', 'HL_4', 'HL_5', 'HL_6'])
    return X, y

# 2. Построение графа совместной встречаемости
def build_cooccurrence_graph(y):
    y_np = y.values
    graph_builder = LabelCooccurrenceGraphBuilder(weighted=True, include_self_edges=False)
    edge_map = graph_builder.transform(y_np)
    n_labels = y_np.shape[1]
    adj_matrix = np.zeros((n_labels, n_labels), dtype=np.float64)
    for (i, j), weight in edge_map.items():
        adj_matrix[i, j] = weight
    return nx.from_numpy_array(adj_matrix)

# 3. Кластеризация меток
def get_label_clusters(graph):
    partition = community_louvain.best_partition(graph, resolution=1.35)
    clusters = {}
    for node, cluster_id in partition.items():
        clusters.setdefault(cluster_id, []).append(node)
    return list(clusters.values())

# 4. Обучение модели
def train_model(X_train, y_train, clusters):
    base_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    clusterer = FixedClusterer(clusters)  # Используем кастомный кластеризатор
    return LabelSpacePartitioningClassifier(
        classifier=LabelPowerset(classifier=base_classifier),
        clusterer=clusterer
    ).fit(X_train, y_train)

In [100]:
X, y = load_data("../0. Data/wide_data.feather")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
graph = build_cooccurrence_graph(y_train)
clusters = get_label_clusters(graph)
model = train_model(X_train, y_train.values, clusters)  # Преобразуем y_train в numpy array
accuracy = model.score(X_test, y_test)
print(f'Model Accuracy: {accuracy:.4f}')

Model Accuracy: 0.0882


In [120]:
answ = model.predict(X_test)

print(answ)

<List of Lists sparse matrix of dtype 'int32'
	with 204 stored elements and shape (68, 6)>
  Coords	Values
  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (1, 1)	1
  (1, 2)	1
  (1, 4)	1
  (1, 5)	1
  (2, 1)	1
  (2, 5)	1
  (3, 1)	1
  (3, 3)	1
  (3, 4)	1
  (3, 5)	1
  (4, 0)	1
  (4, 1)	1
  (4, 2)	1
  (4, 5)	1
  (5, 0)	1
  (5, 1)	1
  (5, 2)	1
  (5, 3)	1
  (6, 1)	1
  (6, 2)	1
  (6, 4)	1
  :	:
  (60, 1)	1
  (60, 3)	1
  (60, 5)	1
  (61, 2)	1
  (61, 5)	1
  (62, 0)	1
  (62, 1)	1
  (62, 2)	1
  (62, 4)	1
  (63, 0)	1
  (63, 1)	1
  (63, 2)	1
  (64, 1)	1
  (64, 2)	1
  (64, 4)	1
  (65, 0)	1
  (65, 1)	1
  (65, 2)	1
  (65, 4)	1
  (66, 1)	1
  (66, 5)	1
  (67, 0)	1
  (67, 1)	1
  (67, 3)	1
  (67, 5)	1


## Попытка решить как задачу регрессии

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr
import networkx as nx
from regr_metrics_func import df_metric, prediction_correction, calc_C_index, my_rmse


# 1. Загрузка и подготовка данных для регрессии
def load_data():
    data = pd.read_feather("../0. Data/wide_data.feather")
    
    # Целевые переменные (предположим, что это числовые показатели)
    y = data[['HL_1', 'HL_2', 'HL_3', 'HL_4', 'HL_5', 'HL_6']]
    
    # Признаки
    X = data.drop(columns=['HL_1', 'HL_2', 'HL_3', 'HL_4', 'HL_5', 'HL_6'])
    
    return X, y

# 2. Построение корреляционного графа целевых переменных
def build_correlation_graph(y):
    # Вычисляем корреляцию Спирмена
    corr_matrix, _ = spearmanr(y)
    
    # Создаем взвешенный граф
    graph = nx.from_numpy_array(corr_matrix)
    return graph

# 3. Кластеризация целевых переменных
def get_target_clusters(graph, resolution=1.0):
    # Используем алгоритм Лувена с настраиваемым разрешением
    partition = community_louvain.best_partition(graph, resolution=resolution, random_state=42)
    
    clusters = {}
    for node, cluster_id in partition.items():
        clusters.setdefault(cluster_id, []).append(node)
    return list(clusters.values())

# 4. Обучение модели регрессии
def train_regression_model(X_train, y_train, clusters):
    base_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # Если есть кластеры, используем группировку
    if clusters:
        models = []
        for cluster in clusters:
            # Обучаем отдельную модель на группу связанных целей
            cluster_regressor = MultiOutputRegressor(base_regressor)
            cluster_regressor.fit(X_train, y_train.iloc[:, cluster])
            models.append(cluster_regressor)
        return models
    else:
        # Обучаем единую модель на все цели
        return MultiOutputRegressor(base_regressor).fit(X_train, y_train)


# 5. Предсказание и оценка
def evaluate_model(models, X_test, y_test, clusters):
    if clusters:
        # Собираем предсказания по кластерам
        y_pred = np.zeros_like(y_test)
        for model, cluster in zip(models, clusters):
            y_pred[:, cluster] = model.predict(X_test)
    else:
        y_pred = models.predict(X_test)
    
    df_metric(prediction_correction(y_pred), y_test, calc_C_index), df_metric(prediction_correction(y_pred), y_test, my_rmse)
    df_metric(y_pred, y_test, calc_C_index), df_metric(y_pred, y_test, my_rmse)

    print(f"Corrected: {}")

    print(df_metric(prediction_correction(y_pred), y_test, calc_C_index), df_metric(prediction_correction(y_pred), y_test))
    print(df_metric(y_pred, y_test, calc_C_index), df_metric(y_pred, y_test))

    # Вычисляем метрики
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R^2 Score: {r2:.4f}")
    return y_pred

In [147]:
# Загрузка данных
X, y = load_data()

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Построение корреляционного графа
graph = build_correlation_graph(y_train)

# Кластеризация целей
clusters = get_target_clusters(graph, resolution=0.5)
print(f"Found {len(clusters)} target clusters")

# Обучение модели
model = train_regression_model(X_train, y_train, clusters)

# Оценка
y_pred = evaluate_model(model, X_test, y_test, clusters)

Found 5 target clusters
Mean Squared Error: 5.0281
R^2 Score: 0.0696


In [148]:
clusters

[[0], [1], [2, 4], [3], [5]]